In [1]:
import pandas as pd
import swifter
import re
import string
from nltk.tag import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer
pd.set_option('display.max_colwidth', None)

In [2]:
tokenized_tagged = pd.read_csv('tokenized_tagged.csv', index_col=0)

In [3]:
tokenized_tagged

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"['NC', 'and', 'NH']",False,"[('NC', 'NNP'), ('and', 'CC'), ('NH', 'NNP')]"
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"['You', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams', 'more', 'than', 'east', 'teams', 'right']",False,"[('You', 'PRP'), ('do', 'VBP'), ('know', 'VB'), ('west', 'JJS'), ('teams', 'NNS'), ('play', 'VBP'), ('against', 'IN'), ('west', 'JJ'), ('teams', 'NNS'), ('more', 'RBR'), ('than', 'IN'), ('east', 'JJ'), ('teams', 'NNS'), ('right', 'RB')]"
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"['They', 'were', 'underdogs', 'earlier', 'today', 'but', 'since', 'Gronks', 'announcement', 'this', 'afternoon', 'the', 'Vegas', 'line', 'has', 'moved', 'to', 'patriots', '1']",False,"[('They', 'PRP'), ('were', 'VBD'), ('underdogs', 'NNS'), ('earlier', 'RBR'), ('today', 'NN'), ('but', 'CC'), ('since', 'IN'), ('Gronks', 'NNP'), ('announcement', 'NN'), ('this', 'DT'), ('afternoon', 'NN'), ('the', 'DT'), ('Vegas', 'NNP'), ('line', 'NN'), ('has', 'VBZ'), ('moved', 'VBN'), ('to', 'TO'), ('patriots', 'VB'), ('1', 'CD')]"
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"['This', 'meme', 'isnt', 'funny', 'none', 'of', 'the', 'new', 'york', 'nigga', 'ones', 'are']",False,"[('This', 'DT'), ('meme', 'NN'), ('isnt', 'VB'), ('funny', 'JJ'), ('none', 'NN'), ('of', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('york', 'NN'), ('nigga', 'JJ'), ('ones', 'NNS'), ('are', 'VBP')]"
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"['I', 'could', 'use', 'one', 'of', 'those', 'tools']",False,"[('I', 'PRP'), ('could', 'MD'), ('use', 'VB'), ('one', 'CD'), ('of', 'IN'), ('those', 'DT'), ('tools', 'NNS')]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"['Im', 'sure', 'that', 'Iran', 'and', 'N', 'Korea', 'have', 'the', 'technology', 'to', 'create', 'a', 'pigbirdhuman', 'hybrid', 'virus']",False,"[('Im', 'NNP'), ('sure', 'NN'), ('that', 'WDT'), ('Iran', 'NNP'), ('and', 'CC'), ('N', 'NNP'), ('Korea', 'NNP'), ('have', 'VBP'), ('the', 'DT'), ('technology', 'NN'), ('to', 'TO'), ('create', 'VB'), ('a', 'DT'), ('pigbirdhuman', 'JJ'), ('hybrid', 'JJ'), ('virus', 'NN')]"
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",whatever you do dont vote green,"['whatever'

Now we want to remove all words that are tagged as proper nouns (NNP)

In [4]:
# now add a feature column that indicates whether an emoticon is present in the comment
def contains_emoticon(text):
    # Define a regex pattern for common emoticons
    emoticon_pattern = r"""
        [:;=8xX]          # Eyes: :, ;, =, 8, x, X
        [-~^]?            # Optional nose: -, ~, ^
        [\)DpPoO/\|\\\(\[\]@#*$] # Mouth or other features
    """
    # Compile the pattern with re.VERBOSE for readability
    emoticon_regex = re.compile(emoticon_pattern, re.VERBOSE)

    # Search for the pattern in the text
    return bool(emoticon_regex.search(text))


In [5]:
tokenized_tagged['contains emoticon'] = tokenized_tagged['comment'].apply(lambda comment: contains_emoticon(comment))

In [6]:
tokenized_tagged

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos,contains emoticon
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"['NC', 'and', 'NH']",False,"[('NC', 'NNP'), ('and', 'CC'), ('NH', 'NNP')]",False
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"['You', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams', 'more', 'than', 'east', 'teams', 'right']",False,"[('You', 'PRP'), ('do', 'VBP'), ('know', 'VB'), ('west', 'JJS'), ('teams', 'NNS'), ('play', 'VBP'), ('against', 'IN'), ('west', 'JJ'), ('teams', 'NNS'), ('more', 'RBR'), ('than', 'IN'), ('east', 'JJ'), ('teams', 'NNS'), ('right', 'RB')]",False
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"['They', 'were', 'underdogs', 'earlier', 'today', 'but', 'since', 'Gronks', 'announcement', 'this', 'afternoon', 'the', 'Vegas', 'line', 'has', 'moved', 'to', 'patriots', '1']",False,"[('They', 'PRP'), ('were', 'VBD'), ('underdogs', 'NNS'), ('earlier', 'RBR'), ('today', 'NN'), ('but', 'CC'), ('since', 'IN'), ('Gronks', 'NNP'), ('announcement', 'NN'), ('this', 'DT'), ('afternoon', 'NN'), ('the', 'DT'), ('Vegas', 'NNP'), ('line', 'NN'), ('has', 'VBZ'), ('moved', 'VBN'), ('to', 'TO'), ('patriots', 'VB'), ('1', 'CD')]",False
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"['This', 'meme', 'isnt', 'funny', 'none', 'of', 'the', 'new', 'york', 'nigga', 'ones', 'are']",False,"[('This', 'DT'), ('meme', 'NN'), ('isnt', 'VB'), ('funny', 'JJ'), ('none', 'NN'), ('of', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('york', 'NN'), ('nigga', 'JJ'), ('ones', 'NNS'), ('are', 'VBP')]",False
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"['I', 'could', 'use', 'one', 'of', 'those', 'tools']",False,"[('I', 'PRP'), ('could', 'MD'), ('use', 'VB'), ('one', 'CD'), ('of', 'IN'), ('those', 'DT'), ('tools', 'NNS')]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"['Im', 'sure', 'that', 'Iran', 'and', 'N', 'Korea', 'have', 'the', 'technology', 'to', 'create', 'a', 'pigbirdhuman', 'hybrid', 'virus']",False,"[('Im', 'NNP'), ('sure', 'NN'), ('that', 'WDT'), ('Iran', 'NNP'), ('and', 'CC'), ('N', 'NNP'), ('Korea', 'NNP'), ('have', 'VBP'), ('the', 'DT'), ('technology', 'NN'), ('to', 'TO'), ('create', 'VB'), ('a', 'DT'), ('pigbirdhuman', 'JJ'), ('hybrid', 'JJ'), ('virus', 'NN')]",False
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on cl

In [7]:
tokenized_tagged[tokenized_tagged['contains emoticon'] == 1]

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos,contains emoticon
106,0,lets hope this 'Swindle' is a perpetual check and not a surprise mating ;),BasketReady,The_Donald,2,-1,-1,2016-11,2016-11-23 21:08:11,Checkmate,lets hope this Swindle is a perpetual check and not a surprise mating,"['lets', 'hope', 'this', 'Swindle', 'is', 'a', 'perpetual', 'check', 'and', 'not', 'a', 'surprise', 'mating']",False,"[('lets', 'NNS'), ('hope', 'VBP'), ('this', 'DT'), ('Swindle', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('perpetual', 'JJ'), ('check', 'NN'), ('and', 'CC'), ('not', 'RB'), ('a', 'DT'), ('surprise', 'NN'), ('mating', 'NN')]",True
135,0,"No problem, thank *you* :)",AerMarcus,IronThronePowers,1,-1,-1,2016-10,2016-10-12 01:10:13,"Thank you. Fixed. Travel noted, safe winds.",No problem thank you,"['No', 'problem', 'thank', 'you']",False,"[('No', 'DT'), ('problem', 'NN'), ('thank', 'NN'), ('you', 'PRP')]",True
157,0,Customers never respect stores closing It is always about what they want Most have no clue how that second can totally throw off a closing shift and they really don't care :/,tlsampl,TalesFromRetail,24,24,0,2016-09,2016-09-19 05:53:49,"Sir, we're closed... Tonight, at 10pm, my store closed. I cashed out my last customer. As I handed her the receipt, another guy walks in, past the automatic doors that were put on halfway open to accommodate my last customer. My shift lead is standing by the doors, keys in hand. ""Sir,"" my shift lead calls after him, ""it's 10 o clock, we are closed."" ""Oh, I know, I'll only be a sec!"" Guy keeps walking back towards the pet stuff. My shift lead lets my last customer out, and stands by the doors, and I can see she's fuming. Here comes Special Guy. ""See! I was only a second! Haha!"" Special Guy literally puffs out his chest with pride. I ring up his 15 big cans of dog food, and bag them in his gross smelling bag. I tell him his total, he pays, and he's watching both of us, and we are less than thrilled with him. ""Geez, ladies, lighten up! I was only a second!"" My shift lead shakes her head. ""Yes, but you're cutting into our closing ops, and yes, 5 minutes DOES make a difference."" She unlocks the doors to let him out, bidding him adieu. And his parting words to us? ""I was only a second though! Good night!"" Good lord, dude. We are open 14 hours; you had all damn day! Also, before my head gets bitten off, I'm aware some people work doubles or long shifts; I am one of them (my other job). But there is a grocery store open until midnight, and the mini mart open until 11pm that can see you through. Sorry, just...argh!!!!!",Customers never respect stores closing It is always about what they want Most have no clue how that second can totally throw off a closing shift and they really dont care,"['Customers', 'never', 'respect', 'stores', 'closing', 'It', 'is', 'always', 'about', 'what', 'they', 'want', 'Most', 'have', 'no', 'clue', 'how', 'that', 'second', 'can', 'totally', 'throw', 'off', 'a', 'closing', 'shift', 'and', 'they', 'really', 'dont', 'care']",False,"[('Customers', 'NNS'), ('never', 'RB'), ('respect', 'VBP'), ('stores', 'NNS'), ('closing', 'VBG'), ('It', 'PRP'), ('is', 'VBZ'), ('always', 'RB'), ('about', 'IN'), ('what', 'WP'), ('they', 'PRP'), ('want', 'VBP'), ('Most', 'JJS'), ('have', 'VBP'), ('no', 'DT'), ('clue', 'NN'), ('how', 'WRB'), ('that', 'DT'), ('second', 'JJ'), ('can', 'MD'), ('totally', 'RB'), ('throw', 'VB'), ('off', 'RP'), ('a', 'DT'), ('closing', 'NN'), ('shift', 'NN'), ('and', 'CC'), ('they', 'PRP'), ('really', 'RB'), ('dont', 'VBZ'), ('care', 'NN')]",True
167,0,You say that but I could see a very hefty guerrilla warfare campaign that would make invading Alaska very expensive for any other state to try and wage.,hangoverDOTTED,AskAnAmerican,1,-1,-1,2016-11,2016-11-06 02:26:30,Hit Anchorage and the population is now around 300k and scattered. They're basically done at that point.,You say that but I c

In [8]:
intensity = SentimentIntensityAnalyzer()
tokenized_tagged['comment positive score'] = tokenized_tagged['comment'].apply(lambda comment: intensity.polarity_scores(comment)['pos'])





In [9]:
tokenized_tagged['comment negative score'] = tokenized_tagged['comment'].apply(lambda comment: intensity.polarity_scores(comment)['neg'])

In [10]:
tokenized_tagged['comment neutral score'] = tokenized_tagged['comment'].apply(lambda comment: intensity.polarity_scores(comment)['neu'])


In [11]:
tokenized_tagged['comment compound score'] = tokenized_tagged['comment'].apply(lambda comment: intensity.polarity_scores(comment)['compound'])

In [12]:
tokenized_tagged

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,no punctuation,tokenized,sticky caps,pos,contains emoticon,comment positive score,comment negative score,comment neutral score,comment compound score
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",NC and NH,"['NC', 'and', 'NH']",False,"[('NC', 'NNP'), ('and', 'CC'), ('NH', 'NNP')]",False,0.663,0.000,0.337,0.6037
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,You do know west teams play against west teams more than east teams right,"['You', 'do', 'know', 'west', 'teams', 'play', 'against', 'west', 'teams', 'more', 'than', 'east', 'teams', 'right']",False,"[('You', 'PRP'), ('do', 'VBP'), ('know', 'VB'), ('west', 'JJS'), ('teams', 'NNS'), ('play', 'VBP'), ('against', 'IN'), ('west', 'JJ'), ('teams', 'NNS'), ('more', 'RBR'), ('than', 'IN'), ('east', 'JJ'), ('teams', 'NNS'), ('right', 'RB')]",False,0.156,0.000,0.844,0.3400
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,They were underdogs earlier today but since Gronks announcement this afternoon the Vegas line has moved to patriots 1,"['They', 'were', 'underdogs', 'earlier', 'today', 'but', 'since', 'Gronks', 'announcement', 'this', 'afternoon', 'the', 'Vegas', 'line', 'has', 'moved', 'to', 'patriots', '1']",False,"[('They', 'PRP'), ('were', 'VBD'), ('underdogs', 'NNS'), ('earlier', 'RBR'), ('today', 'NN'), ('but', 'CC'), ('since', 'IN'), ('Gronks', 'NNP'), ('announcement', 'NN'), ('this', 'DT'), ('afternoon', 'NN'), ('the', 'DT'), ('Vegas', 'NNP'), ('line', 'NN'), ('has', 'VBZ'), ('moved', 'VBN'), ('to', 'TO'), ('patriots', 'VB'), ('1', 'CD')]",False,0.000,0.000,1.000,0.0000
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,This meme isnt funny none of the new york nigga ones are,"['This', 'meme', 'isnt', 'funny', 'none', 'of', 'the', 'new', 'york', 'nigga', 'ones', 'are']",False,"[('This', 'DT'), ('meme', 'NN'), ('isnt', 'VB'), ('funny', 'JJ'), ('none', 'NN'), ('of', 'IN'), ('the', 'DT'), ('new', 'JJ'), ('york', 'NN'), ('nigga', 'JJ'), ('ones', 'NNS'), ('are', 'VBP')]",False,0.000,0.179,0.821,-0.3412
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,I could use one of those tools,"['I', 'could', 'use', 'one', 'of', 'those', 'tools']",False,"[('I', 'PRP'), ('could', 'MD'), ('use', 'VB'), ('one', 'CD'), ('of', 'IN'), ('those', 'DT'), ('tools', 'NNS')]",False,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",Im sure that Iran and N Korea have the technology to create a pigbirdhuman hybrid virus,"['Im', 'sure', 'that', 'Iran', 'and', 'N', 'Korea', 'have', 'the', 'technology', 'to', 'create', 'a', 'pigbirdhuman', 'hybrid', 'virus']",False,"[('Im', 'NNP'), ('sure', 'NN'), ('that', 'WDT'), ('Iran', 'NNP'), ('and', 'CC'), ('N', 'NNP'), ('Korea', 'NNP'), ('have', 'VBP'), ('the', 'DT'), ('technology', 'NN'), ('to', 'TO'), ('create', 'VB'), ('a', 'DT'), ('pigbirdhuman', 'JJ'), ('hybrid', 'JJ'), ('virus', 'NN')]",False,0.2

In [13]:
tokenized_tagged['parent comment positive score'] = tokenized_tagged['parent_comment'].apply(lambda comment: intensity.polarity_scores(comment)['pos'])

In [14]:
tokenized_tagged['parent comment negative score'] = tokenized_tagged['parent_comment'].apply(lambda comment: intensity.polarity_scores(comment)['neg'])

In [15]:
tokenized_tagged['parent comment neutral score'] = tokenized_tagged['parent_comment'].apply(lambda comment: intensity.polarity_scores(comment)['neu'])


In [16]:
tokenized_tagged['parent comment compound score'] = tokenized_tagged['parent_comment'].apply(lambda comment: intensity.polarity_scores(comment)['compound'])

In [17]:
tokenized_tagged['compound difference'] = abs(tokenized_tagged['comment compound score'] - tokenized_tagged['parent comment compound score'])

In [18]:
tokenized_tagged

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment,...,contains emoticon,comment positive score,comment negative score,comment neutral score,comment compound score,parent comment positive score,parent comment negative score,parent comment neutral score,parent comment compound score,compound difference
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd prefer is she lived in NC as well.",...,False,0.663,0.000,0.337,0.6037,0.217,0.126,0.657,0.2023,0.4014
1,0,You do know west teams play against west teams more than east teams right?,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 seed) did not even carry a good enough record to make the playoffs in the east last year.,...,False,0.156,0.000,0.844,0.3400,0.000,0.095,0.905,-0.3412,0.6812
2,0,"They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.,...,False,0.000,0.000,1.000,0.0000,0.767,0.000,0.233,0.7650,0.7650
3,0,"This meme isn't funny none of the ""new york nigga"" ones are.",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz,...,False,0.000,0.179,0.821,-0.3412,0.483,0.000,0.517,0.5773,0.9185
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for that. It was made by our boy EASports_MUT,...,False,0.000,0.000,1.000,0.0000,0.121,0.000,0.879,0.2960,0.2960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the technology to create a pig/bird/human hybrid virus.,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen, but there are reports that this virus has bits of human, pig and bird genetic material, which makes it uniquely interesting",...,False,0.253,0.000,0.747,0.5267,0.110,0.050,0.840,0.4497,0.0770
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,"In a move typical of their recent do-nothing approach to CO2 emissions, Canada appoints a tar sands executive to liaise with the US on climate change",...,False,0.000,0.000,1.000,0.0000,0.000,0.000,1.000,0.0000,0.0000
1010823,1,Perhaps this is an atheist conspiracy to make Christians look bad?,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church on Time,...,False,0.000,0.434,0.566,-0.7845,0.000,0.135,0.865,-0.1027,0.6818
1010824,1,The Slavs got their own country - it is called Kosovo,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,"I've always been unsettled by that. I hear a lot of Jewish people say that Israel can do whatever it pleases because of the holocaust, but the Slavs, and other races the Germans attempted to annihilate go unnoticed in history?",...,False,0.000,0.000,1.000,0.0000,0.047,0.042,0.911,0.0516,0.0516


In [19]:
tokenized_tagged.to_csv('all_features.csv')